# Creando dataset

In [1]:
import pandas as pd 
import numpy as np 
import json

Deteccion de microparadas
Flujo:
1. Primero generaremos csv de microparadas para cada objeto
2. Como seran csv entonces ya 

In [2]:
df_1 = pd.read_csv("../BD_Innovathon/Objeto_1_ES_pos.csv")
df_2 = pd.read_csv("../BD_Innovathon/Objeto_2_ES_pos.csv")
print(df_2.head(3))

   Id         Fecha y hora               posición  Distancia (m)  \
0   1  29/09/2025 22:15:14  S12 04.069 W77 03.038              0   
1   2  29/09/2025 22:16:14  S12 04.134 W77 02.957            191   
2   3  29/09/2025 22:16:44  S12 04.137 W77 02.956              5   

  Tiempo de registro (s)  Velocidad (km/h)  
0                0:00:00                 0  
1                0:01:00                11  
2                0:00:30                 1  


In [10]:
def detect_microparadas_by_object(dataframe_object, object_id):
    # for para cada registro    
    dataframe_object['is_microparada'] = dataframe_object['Velocidad (km/h)'].apply(lambda x: 1 if x < 6 else 0)
    ultima_fila = dataframe_object.iloc[-1]
    alerta = bool(ultima_fila['is_microparada'] == 1)

    # Respuesta tipo JSON para la mini app
    result = {
        "objeto_id": object_id,
        "alerta": alerta,
        "fecha_hora": str(ultima_fila['Fecha y hora']),
        "posicion": ultima_fila['posición'],
        "velocidad": ultima_fila['Velocidad (km/h)']
    }
    return result
    

In [11]:
dft_1 = detect_microparadas_by_object(df_1, 1)
print(dft_1)
#dft_2 = detect_microparadas_by_object(df_2, objeto_id=2) 
#print(dft_2.shape)
#dft_2 = detect_microparadas_by_object(df_2, objeto_id=2)
#dft_3 = detect_microparadas_by_object(df_3, objeto_id=3)

{'objeto_id': 1, 'alerta': True, 'fecha_hora': '29/09/2025 23:19:19', 'posicion': 'S11 59.972 W76 49.870', 'velocidad': np.int64(0)}


In [13]:
!pip install scikit-learn

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 1. Cargar datos y detectar microparadas
df = pd.read_csv("../BD_Innovathon/Objeto_1_ES_pos.csv")
df['is_microparada'] = df['Velocidad (km/h)'].apply(lambda v: 1 if v < 6 else 0)
df['hora'] = pd.to_datetime(df['Fecha y hora'], dayfirst=True).dt.hour
df['dia_semana'] = pd.to_datetime(df['Fecha y hora'], dayfirst=True).dt.dayofweek


# 2. Generar etiquetas (congestión si hay muchas microparadas en un lugar/hora)
trafico = df.groupby(['posición','hora']).agg({'is_microparada':'sum'}).reset_index()
trafico['y'] = (trafico['is_microparada'] >= 3).astype(int)

# 3. Variables predictoras
X = trafico[['hora']]
y = trafico['y']

# 4. Modelo simple
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# 5. Predicción
predicciones = modelo.predict(X_test)



In [18]:
print(predicciones)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [19]:
# Analizar los datos para entender mejor el problema
print("Análisis de datos:")
print(f"Total de microparadas: {df['is_microparada'].sum()}")
print(f"Porcentaje de microparadas: {df['is_microparada'].mean()*100:.2f}%")
print("\nDistribución por hora:")
print(df.groupby('hora')['is_microparada'].agg(['count', 'sum', 'mean']))

print("\nDatos de tráfico agrupados:")
print(trafico.head(10))
print(f"\nEtiquetas de congestión (y=1): {trafico['y'].sum()}")
print(f"Total de observaciones: {len(trafico)}")

Análisis de datos:
Total de microparadas: 25
Porcentaje de microparadas: 30.12%

Distribución por hora:
      count  sum      mean
hora                      
22       62   18  0.290323
23       21    7  0.333333

Datos de tráfico agrupados:
                posición  hora  is_microparada  y
0  S11 59.937 W76 50.025    23               0  0
1  S11 59.938 W76 50.023    23               1  0
2  S11 59.938 W76 50.028    23               0  0
3  S11 59.972 W76 49.870    23               3  1
4  S11 59.972 W76 49.872    23               3  1
5  S11 59.974 W76 49.878    23               0  0
6  S11 59.975 W76 49.869    23               0  0
7  S12 00.021 W76 49.986    23               0  0
8  S12 00.021 W76 49.989    23               0  0
9  S12 00.029 W76 49.994    23               0  0

Etiquetas de congestión (y=1): 3
Total de observaciones: 73


In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd

def pos_to_latlon(pos_str):
    """Convierte 'S12 01.393 W76 54.690' a lat/lon decimales."""
    if pd.isna(pos_str):
        return None, None
    try:
        parts = pos_str.replace("O", "W").split()
        hemi_lat, deg_lat, min_lat = parts[0][0], float(parts[0][1:]), float(parts[1])
        hemi_lon, deg_lon, min_lon = parts[2][0], float(parts[2][1:]), float(parts[3])
        lat = deg_lat + min_lat/60
        lon = deg_lon + min_lon/60
        if hemi_lat.upper() == "S": lat = -lat
        if hemi_lon.upper() == "W": lon = -lon
        return round(lat, 6), round(lon, 6)
    except:
        return None, None

def preparar_csv(input_path, output_path):
    df = pd.read_csv(input_path, encoding="utf-8-sig")

    # Lat/Lon
    df[["Latitud", "Longitud"]] = df["posición"].apply(lambda x: pd.Series(pos_to_latlon(x)))

    # Descomponer fecha y hora
    dt = pd.to_datetime(df["Fecha y hora"], dayfirst=True, errors="coerce")
    df["Año"] = dt.dt.year
    df["Mes"] = dt.dt.month
    df["Día"] = dt.dt.day
    df["Hora"] = dt.dt.hour
    df["Minuto"] = dt.dt.minute
    df["Segundo"] = dt.dt.second

    # Marcar microparadas
    df["is_microparada"] = df["Velocidad (km/h)"].apply(lambda x: 1 if x < 6 else 0)

    # Nos quedamos SOLO con las columnas procesadas
    df_final = df[["Id", "Latitud", "Longitud", "Año", "Mes", "Día", "Hora", "Minuto", "Segundo", "Velocidad (km/h)", "is_microparada"]]

    # Guardar
    df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"[OK] Archivo generado: {output_path}")



# Ejemplo de uso
if __name__ == "__main__":
    preparar_csv("../BD_Innovathon/Objeto_2_ES_pos.csv", "../BD_Innovathon/Objeto_2_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_3_ES_pos.csv", "../BD_Innovathon/Objeto_3_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_4_ES_pos.csv", "../BD_Innovathon/Objeto_4_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_5_ES_pos.csv", "../BD_Innovathon/Objeto_5_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_6_ES_pos.csv", "../BD_Innovathon/Objeto_6_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_7_ES_pos.csv", "../BD_Innovathon/Objeto_7_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_8_ES_pos.csv", "../BD_Innovathon/Objeto_8_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_9_ES_pos.csv", "../BD_Innovathon/Objeto_9_ES_pos_num.csv")
    preparar_csv("../BD_Innovathon/Objeto_10_ES_pos.csv", "../BD_Innovathon/Objeto_10_ES_pos_num.csv")



[OK] Archivo generado: ../BD_Innovathon/Objeto_2_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_3_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_4_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_5_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_6_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_7_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_8_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_9_ES_pos_num.csv
[OK] Archivo generado: ../BD_Innovathon/Objeto_10_ES_pos_num.csv
